In [5]:
import cv2
import numpy as np

# Задания
1. Для данных Nails segmentation объедините пары изображение-маска (список файлов получить с помощью библиотеки os название парных изображений совпадают)
2. Выведите по очереди пары с помощью OpenCV эти пары (переключение по нажатию клавиши)
3. Выделите контуры на масках и отрисуйте их на изображениях
4. Воспроизведите любой видеофайл с помощью OpenCV в градациях серого

In [6]:
import os

In [7]:
images_dir = r'C:\Users\Ilya\Tasks PAK\data\nails_segmentation\images'
labels_dir = r'C:\Users\Ilya\Tasks PAK\data\nails_segmentation\labels'

images_files = [file for file in os.listdir(images_dir)]

images_path = [os.path.join(images_dir, file) for file in images_files]
labels_path = [os.path.join(labels_dir, file) for file in images_files]


## Task1

In [8]:
new_dir = 'nails_segmentation/images_and_labels'

In [9]:
img1 = cv2.imread(images_path[1])
type(img1)

numpy.ndarray

In [10]:
for i in range(len(images_path)) :
    img = cv2.imread(images_path[i])
    label = cv2.imread(labels_path[i])

    img_and_label = cv2.bitwise_and(img, label)

    cv2.imshow('Image and Label', img_and_label)
    cv2.imwrite(os.path.join(new_dir,images_files[i]), img_and_label)
    key = cv2.waitKey(0) & 0xff
    
    if key == 27:
        break
    cv2.destroyWindow('Image and Label')
    
cv2.destroyWindow('Image and Label')

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:1261: error: (-27:Null pointer) NULL window: 'Image and Label' in function 'cvDestroyWindow'


## Task2

In [11]:
for i in range(len(images_path)) :
    img = cv2.imread(images_path[i])
    label = cv2.imread(labels_path[i])

    cv2.imshow('Image', img)
    cv2.imshow('Label', label)
    key = cv2.waitKey(0) & 0xff
    
    if key == 27:
        break
    cv2.destroyAllWindows()

cv2.destroyAllWindows()        

## Task3

In [31]:
for i in range(len(images_path)) :
    img = cv2.imread(images_path[i])
    label = cv2.imread(labels_path[i], 0)

    label = cv2.GaussianBlur(label, (5, 5), 0)
    
    contours, hierarchy = cv2.findContours(label, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    img = cv2.drawContours(img, contours, -1, (0, 255, 0), 2)
    
    cv2.imshow('Image with contours', img)
    key = cv2.waitKey(0) & 0xff
    
    if key == 27:
        break
    cv2.destroyWindow('Image with contours') 
cv2.destroyWindow('Image with contours')

## Task4

In [32]:
camera = cv2.VideoCapture(0)
while True:
    ret, frame = camera.read()
    key = cv2.waitKey(20) & 0xff
    frame = cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
    cv2.imshow('Frame', frame)
    if key == 27: 
        break
        
cv2.destroyWindow('Frame')
camera.release()

# Лабораторная работа

1. Покадровое получение видеопотока с камеры. Использовать камеру ноутбука, вебкамеру или записать видео файл с вебкамеры товарища и использовать его.
2. Реализовать обнаружение движения в видеопотоке: попарно сравнивать текущий и предыдущий кадры.
3. По мере проигрывания видео в отдельном окне отрисовывать двухцветную карту с результатом: красное - есть движение, зелёное - нет движения
4. Добавить таймер, по которому включается и выключается обнаружение движения. О текущем режиме программы сообщать текстом с краю изображения: “Красный свет” - движение обнаруживается, “Зелёный свет” - движение не обнаруживается.
5. Реализовать более сложный алгоритм обнаружения движения, устойчивый к шумам вебкамеры (OpticalFlow)

In [2]:
import time

In [3]:
def detection_timer(start_time,current_time,detection_enabled):
    if current_time - start_time > 10.0 :
        start_time = current_time
        detection_enabled = not detection_enabled
    return start_time, detection_enabled

In [4]:
camera = cv2.VideoCapture(0)

 # установка размера окна
camera.set(3,500)
camera.set(4,300)

start_time = time.time()
detection_enabled = True


_, frame = camera.read()
frame = cv2.GaussianBlur(frame, (5,5), 0)
_, newFrame = camera.read()

while cv2.waitKey(20) & 0xff != 27:

    # Проверяем таймер для включения/выключения обнаружения движения
    current_time = time.time()
    start_time, detection_enabled = detection_timer(start_time, current_time, detection_enabled)
    
    newFrame = cv2.GaussianBlur(newFrame, (5,5), 0)

    diff_frame = cv2.absdiff(frame, newFrame) 
    
    gray_frame = cv2.cvtColor(diff_frame,cv2.COLOR_RGB2GRAY)

    blur_frame = cv2.GaussianBlur(gray_frame, (5,5), 0)

    _, thresh = cv2.threshold(blur_frame, 20, 255, cv2.THRESH_BINARY)

    dilated = cv2.dilate(thresh, None, iterations = 3)


    start_time_str = time.strftime("%S", time.localtime(10-(current_time-start_time)))

    green_red_dilated = np.zeros((dilated.shape[0], dilated.shape[1], 3), dtype=np.uint8)
    green_red_dilated[dilated==255] = [0,0,255]
    green_red_dilated[dilated==0] = [0,255,0]


    if detection_enabled:
        cv2.putText(frame, 'Red light', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.putText(frame, start_time_str, (370, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.putText(frame, 'seconds before green:', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.putText(green_red_dilated, 'detection_enabled', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.imshow('motion detection', green_red_dilated)
    else:
        if cv2.getWindowProperty('motion detection', cv2.WND_PROP_VISIBLE) > 0:
            cv2.destroyWindow('motion detection')
        cv2.putText(frame, 'Green light', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(frame, start_time_str, (350, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(frame, 'seconds before red:', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        

    cv2.imshow('Frame', frame)
        
    frame = newFrame
    _, newFrame = camera.read()
    

cv2.destroyAllWindows()
camera.release()